<a href="https://colab.research.google.com/github/mario-97/Challenge_DataScientist/blob/desarrollo/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar librerías necesarias

In [1]:
from google.colab import files
import pandas as pd
import io 
from collections import Counter

# SUBIDA Y LECTURA DE ARCHIVO .CSV

Subir archivo .csv

In [2]:
uploaded = files.upload(); 
dataset_SCL = pd.read_csv(io.BytesIO(uploaded['dataset_SCL.csv'])) # selección del archivo

Saving dataset_SCL.csv to dataset_SCL.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
print(dataset_SCL) # imprimir dataframe

                   Fecha-I Vlo-I Ori-I Des-I Emp-I              Fecha-O  \
0      2017-01-01 23:30:00   226  SCEL  KMIA   AAL  2017-01-01 23:33:00   
1      2017-01-02 23:30:00   226  SCEL  KMIA   AAL  2017-01-02 23:39:00   
2      2017-01-03 23:30:00   226  SCEL  KMIA   AAL  2017-01-03 23:39:00   
3      2017-01-04 23:30:00   226  SCEL  KMIA   AAL  2017-01-04 23:33:00   
4      2017-01-05 23:30:00   226  SCEL  KMIA   AAL  2017-01-05 23:28:00   
...                    ...   ...   ...   ...   ...                  ...   
68201  2017-12-22 14:55:00   400  SCEL  SPJC   JAT  2017-12-22 15:41:00   
68202  2017-12-25 14:55:00   400  SCEL  SPJC   JAT  2017-12-25 15:11:00   
68203  2017-12-27 14:55:00   400  SCEL  SPJC   JAT  2017-12-27 15:35:00   
68204  2017-12-29 14:55:00   400  SCEL  SPJC   JAT  2017-12-29 15:08:00   
68205  2017-12-31 14:55:00   400  SCEL  SPJC   JAT  2017-12-31 15:04:00   

       Vlo-O Ori-O Des-O Emp-O  DIA  MES   AÑO     DIANOM TIPOVUELO  \
0        226  SCEL  KMIA   A

Obtener dimensiones del dataframe

In [4]:
dim = dataset_SCL.shape
dim # mostrar dimensiones

(68206, 18)

Convertir columnas 'Fecha-O' y 'Fecha-I' en formato fecha

In [5]:
dataset_SCL['Fecha-O'] = pd.to_datetime(dataset_SCL['Fecha-O'])
dataset_SCL['Fecha-I'] = pd.to_datetime(dataset_SCL['Fecha-I'])


# CALCULO DE LA TEMPORADA ALTA

Primero que todo, para calcular la temporada alta, debemos separar todos los periodos que correspondan, en este caso: entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, respecto a la columna 'Fecha-I'

In [7]:
# generar periodos de alta temporada
from datetime import datetime # libreria 

# año a evaluar.
año = dataset_SCL['Fecha-I'][0].year
# EN CASO DE QUE SE ESTE CONSIDERANDO MAS DE UN AÑO EN EL DATASET, ESTA SECCION DE CODIGO DEBE INGRESARSE DENTRO DEL FOR DE LA CELDA SUBSIGUIENTE (CAMBIANDO EL VALOR [0] POR [i])

# para la fecha 15/12/2017
temp_1_inicio = datetime.strptime('15/12/'+str(año), '%d/%m/%Y')
print(temp_1_inicio)
 
# para la fecha 03/03/2017
temp_1_fin = datetime.strptime('3/3/'+str(año), '%d/%m/%Y')
print(temp_1_fin)

# para la fecha 15/07/2017
temp_2_inicio = datetime.strptime('15/7/'+str(año), '%d/%m/%Y')
print(temp_2_inicio)
 
# para la fecha 31/07/2017
temp_2_fin = datetime.strptime('31/7/'+str(año), '%d/%m/%Y')
print(temp_2_fin)

# para la fecha 11/09/2017
temp_3_inicio = datetime.strptime('11/9/'+str(año), '%d/%m/%Y')
print(temp_3_inicio)
 
# para la fecha 30/09/2017
temp_3_fin = datetime.strptime('30/9/'+str(año), '%d/%m/%Y')
print(temp_3_fin)

2017-12-15 00:00:00
2017-03-03 00:00:00
2017-07-15 00:00:00
2017-07-31 00:00:00
2017-09-11 00:00:00
2017-09-30 00:00:00


Con las temporadas ya obtenidas, simplemente se recorre la columna 'Fecha-I', buscando si la fecha corresponde a una temporada alta o no. Esta informacion se guarda en una lista binaria 'temporada', en donde tenemos: 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.

In [8]:
# temporada_alta : 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
temporada = []
for i in dataset_SCL['Fecha-I']: 
  if temp_2_inicio < i < temp_2_fin or temp_2_inicio < i < temp_2_fin or temp_1_inicio < i or i < temp_1_fin: 
    temporada.append(1) 
  else:
    temporada.append(0) 

Se convierte la lista 'temporada' a dataframe, y se concatena con el dataframe 'dataset_SCL'

In [9]:
temporada = pd.DataFrame(temporada) # convertir lista a dataframe
dataset_SCL = pd.concat([dataset_SCL, temporada], axis=1) # concatenacion
dataset_SCL = dataset_SCL.rename(columns={0:'temporada_alta'}) # renombrar columna concatenada a 'temporada_alta'
dataset_SCL # mostrar

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,temporada_alta
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami,1
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami,1
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami,1
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami,1
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,22,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,25,12,2017,Lunes,I,JetSmart SPA,Santiago,Lima,1
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,27,12,2017,Miercoles,I,JetSmart SPA,Santiago,Lima,1
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,29,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1


# CALCULO DE DIFERENCIA EN MINUTOS

Obtener diferencia de fechas

In [10]:
diferencia_min = dataset_SCL['Fecha-O'] - dataset_SCL['Fecha-I']
diferencia_min # mostrar diferencia de fechas en formato fecha

0         0 days 00:03:00
1         0 days 00:09:00
2         0 days 00:09:00
3         0 days 00:03:00
4       -1 days +23:58:00
               ...       
68201     0 days 00:46:00
68202     0 days 00:16:00
68203     0 days 00:40:00
68204     0 days 00:13:00
68205     0 days 00:09:00
Length: 68206, dtype: timedelta64[ns]

Obtener los minutos de estas diferencias de fecha

In [11]:
for i in range(diferencia_min.shape[0]):
  diferencia_min[i] = diferencia_min[i].total_seconds() / 60
diferencia_min 

0         3.0
1         9.0
2         9.0
3         3.0
4        -2.0
         ... 
68201    46.0
68202    16.0
68203    40.0
68204    13.0
68205     9.0
Length: 68206, dtype: object

Concatenar la columna 'diferencia_min' al dataframe 'dataset_SCL'

In [12]:
dataset_SCL = pd.concat([dataset_SCL, diferencia_min], axis=1) # concatenación
dataset_SCL = dataset_SCL.rename(columns={0:'dif_min'}) # renombrar columna concatenada
dataset_SCL # mostrar 

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,temporada_alta,dif_min
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami,1,3.0
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami,1,9.0
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami,1,9.0
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami,1,3.0
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami,1,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,22,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1,46.0
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,25,12,2017,Lunes,I,JetSmart SPA,Santiago,Lima,1,16.0
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,27,12,2017,Miercoles,I,JetSmart SPA,Santiago,Lima,1,40.0
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,29,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1,13.0


# CALCULO DE PERIODOS DIA (MAÑANA, TARDE Y NOCHE)

Para la obtención de los períodos del dia, se recorre la columna de 'Fecha-I', buscando en cada fecha, los intervalos de tiempo (en hora) para determinar si esta corresponde a: mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59). Y asignar este periodo a una lista

In [13]:
periodos = [] # crear lista vacia
for i in range(dim[0]):
  if dataset_SCL['Fecha-I'][i].hour > 5 and dataset_SCL['Fecha-I'][i].hour < 12: # condicion para el periodo de mañana
    periodos.append('mañana')
  elif dataset_SCL['Fecha-I'][i].hour > 12 and dataset_SCL['Fecha-I'][i].hour < 18: # condicion para el periodo de tarde
    periodos.append('tarde')
  else: #dataset_SCL['Fecha-I'][i].hour > 18 and dataset_SCL['Fecha-I'][i].hour < 5: # condicion para el periodo de noche
    periodos.append('noche') 
Counter(periodos) 

Counter({'mañana': 22838, 'noche': 27436, 'tarde': 17932})

Con los periodos del dia obtenidos. La lista periodo_dia se convierte en dataframe, y esta de concatena con el dataframe 'dataset_SCL'

In [14]:
periodo_dia = pd.DataFrame(periodos) # convertir lista en dataframe
dataset_SCL = pd.concat([dataset_SCL, periodo_dia], axis=1) # unir dataframe's
dataset_SCL = dataset_SCL.rename(columns={0:'periodo_dia'}) # renombrar columna del dataframe concatenado
dataset_SCL # mostrar

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,temporada_alta,dif_min,periodo_dia
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,1,2017,Domingo,I,American Airlines,Santiago,Miami,1,3.0,noche
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,1,2017,Lunes,I,American Airlines,Santiago,Miami,1,9.0,noche
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,1,2017,Martes,I,American Airlines,Santiago,Miami,1,9.0,noche
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,1,2017,Miercoles,I,American Airlines,Santiago,Miami,1,3.0,noche
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,1,2017,Jueves,I,American Airlines,Santiago,Miami,1,-2.0,noche
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1,46.0,tarde
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,12,2017,Lunes,I,JetSmart SPA,Santiago,Lima,1,16.0,tarde
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,12,2017,Miercoles,I,JetSmart SPA,Santiago,Lima,1,40.0,tarde
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1,13.0,tarde


# CALCULO DE ATRASOS DE MAS DE 15 MINUTOS

Para calcular los atrasos de mas de 15 minutos, se recorre la columna 'dif_min', buscando aquellos minutos sean mayor a 15. Y asignar esta informacion a una lista binaria ('atraso'), siendo > 15 = 1, caso contrario = 0

In [15]:
atraso = [] # lista
for i in dataset_SCL['dif_min']: # recorrer columna 'dif_min' del dataset
  if i > 15: # condicion minutos mayores a 15
    atraso.append(1) 
  else:
    atraso.append(0)  

Counter(atraso)

Counter({0: 55592, 1: 12614})

Converir lista de 'atraso' en dataframe para concatenerla con el dataframe 'dataset_SCL'

In [16]:
atraso = pd.DataFrame(atraso) # convertir lista a dataframe
dataset_SCL = pd.concat([dataset_SCL, atraso], axis=1) # concatenacion
dataset_SCL = dataset_SCL.rename(columns={0:'atraso_15'}) # renombrar columna concatenada a 'atraso_15
dataset_SCL # mostrar

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,temporada_alta,dif_min,periodo_dia,atraso_15
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,2017,Domingo,I,American Airlines,Santiago,Miami,1,3.0,noche,0
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,2017,Lunes,I,American Airlines,Santiago,Miami,1,9.0,noche,0
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,2017,Martes,I,American Airlines,Santiago,Miami,1,9.0,noche,0
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,2017,Miercoles,I,American Airlines,Santiago,Miami,1,3.0,noche,0
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,2017,Jueves,I,American Airlines,Santiago,Miami,1,-2.0,noche,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1,46.0,tarde,1
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,2017,Lunes,I,JetSmart SPA,Santiago,Lima,1,16.0,tarde,1
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,2017,Miercoles,I,JetSmart SPA,Santiago,Lima,1,40.0,tarde,1
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1,13.0,tarde,0


# EXPORTACION DE COLUMNAS NUEVAS

In [17]:
synthetic_features = dataset_SCL[['temporada_alta','dif_min',	'atraso_15', 'periodo_dia']] # Filtrado de columnas del dataset_SCL para ser asignado a la variable synthetic_features
synthetic_features.to_csv('synthetic_features.csv') # creacion del archivo .CSV
files.download('synthetic_features.csv') # descargar del archivo .CSV

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
synthetic_features # mostrar

,temporada_alta,dif_min,atraso_15,periodo_dia
0,1,3.0,0,noche
1,1,9.0,0,noche
2,1,9.0,0,noche
3,1,3.0,0,noche
4,1,-2.0,0,noche
...,...,...,...,...
68201,1,46.0,1,tarde
68202,1,16.0,1,tarde
68203,1,40.0,1,tarde
68204,1,13.0,0,tarde


# TASAS DE ATRASO

¿Cómo se compone la tasa de atraso por destino, aerolínea, mes del año, día de la semana, temporada, tipo de vuelo?
¿Qué variables esperarías que más influyeran en predecir atrasos?

In [19]:
tasas_atraso = dataset_SCL[['SIGLADES', 'OPERA', 'MES', 'DIANOM', 'temporada_alta', 'TIPOVUELO', 'atraso_15']]

In [20]:
print("-------------TASA DE ATRASO POR CIUDAD DE DESTINO-------------")
print((tasas_atraso.groupby(['SIGLADES']).mean()*100)['atraso_15']) # se agrupan los datos de 'siglades' y se calcula el porcentaje de atraso respecto al total de vuelos
print("-------------TASA DE ATRASO POR AEROLINEA-------------")
print((tasas_atraso.groupby(['OPERA']).mean()*100)['atraso_15'])
print("-------------TASA DE ATRASO POR MES-------------")
print((tasas_atraso.groupby(['MES']).mean()*100)['atraso_15'])
print("-------------TASA DE ATRASO POR DIA DE LA SEMANA-------------")
print((tasas_atraso.groupby(['DIANOM']).mean()*100)['atraso_15'])
print("-------------TASA DE ATRASO POR TEMPORADA ALTA-------------")
print((tasas_atraso.groupby(['temporada_alta']).mean()*100)['atraso_15'])
print("-------------TASA DE ATRASO POR TIPO DE VUELO-------------")
print((tasas_atraso.groupby(['TIPOVUELO']).mean()*100)['atraso_15'])

# Ejemplo: Antofagasta tiene un 14.6% de atrasos (respecto al total de vuelos registrados) de mas de 15 min
# Ejemplo 2: los dias domingos tienen un 16.1% de atrasos (respecto al total de vuelos registrados) de mas de 15 min

-------------TASA DE ATRASO POR CIUDAD DE DESTINO-------------
SIGLADES
Antofagasta      14.638783
Arica            13.546798
Asuncion         28.448276
Atlanta           5.865922
Auckland N.Z.    37.396122
                   ...    
Toronto          31.448763
Tucuman          26.315789
Ushuia           66.666667
Valdivia         15.537849
Washington        0.000000
Name: atraso_15, Length: 62, dtype: float64
-------------TASA DE ATRASO POR AEROLINEA-------------
OPERA
Aerolineas Argentinas       24.012314
Aeromexico                   2.849003
Air Canada                  45.663717
Air France                   8.379888
Alitalia                    11.196911
American Airlines            6.737120
Austral                     20.270270
Avianca                     26.562500
British Airways             14.146341
Copa Air                     6.270270
Delta Air                    5.865922
Gol Trans                   35.607940
Grupo LATAM                 17.866575
Iberia                       9.9

In [27]:
# INICIO BLOQUE DE CODIGO DE COMPROBACION DE PORCENTAJES
evaluar = 'TIPOVUELO'
freq_tasa = tasas_atraso.groupby(['atraso_15', evaluar]).size() # tasa de atraso y no atrasados por cada dia de la semana

freq_total = tasas_atraso.groupby([evaluar]).size() # dias totales
print("Numero total de vuelos(por "+evaluar+"): ")
print(freq_total)
print("Numero de vuelos con atraso(por "+evaluar+"): ")
print(freq_tasa[1]) # solo vuelvos atrasados(1) por dia

for i in range(freq_total.shape[0]): 
  print("porcentaje de vuelos atrasados: "+str(freq_tasa[1][i]*100/freq_total[i])+"%")
# FIN CODIGO DE COMPROBACION DE PORCENTAJES

Numero total de vuelos(en TIPOVUELO): 
TIPOVUELO
I    31240
N    36966
dtype: int64
Numero de vuelos con atraso(en TIPOVUELO): 
TIPOVUELO
I    7048
N    5566
dtype: int64
porcentaje de vuelos atrasados: 22.56081946222791%
porcentaje de vuelos atrasados: 15.057079478439647%
